<a href="https://colab.research.google.com/github/khaichiong/meco7312/blob/master/Lecture7_GMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [117]:
import numpy as np
from scipy.optimize import minimize_scalar
from numpy.linalg import inv

#Optimal GMM

Vector of moment conditions ${\bf h}(\theta,X)$ such that $E[{\bf h}(\theta,X)] = \vec{0}$

In [293]:
def h(theta,x):
  return np.array([[x - theta/2],
            [x**2 - (theta**2)/3],
            [x**3 - (theta**3)/4]])

GMM criterion function with equal weight (inefficient but consistent):

 $Q(\theta) =  \left(\frac{1}{n}\sum_{i=1}^{n}	{\bf h}(\theta,X_{i})\right)^{T}  \left(\frac{1}{n}\sum_{i=1}^{n}	{\bf h}(\theta,X_{i})\right)$

In [294]:
def Q(theta, data):
  hx=0
  for x_data in data:
    hx += h(theta,x_data)/len(data)
  return np.matmul(np.transpose(hx),hx)[0][0]

Suppose we observe the following data

In [295]:
data = np.array([0.1,0.2,0.6,1.0])

Method of moments estimator:

In [296]:
np.mean(data)*2

0.95

GMM estimator with identity weighting matrix. Minimizing the criterion function

In [297]:
res = minimize_scalar(Q,args = data)

In [298]:
res

     fun: 0.002671444829637783
    nfev: 16
     nit: 12
 success: True
       x: 1.034208326372224

## Optimal weighting matrix

In [299]:
theta_hat = res.x
S = 0
for x_data in data: 
  S += np.matmul(h(theta_hat,x_data),np.transpose(h(theta_hat,x_data)))/len(data)
S

array([[0.12864776, 0.13898214, 0.1361055 ],
       [0.13898214, 0.15858498, 0.16144969],
       [0.1361055 , 0.16144969, 0.16877364]])

In [300]:
#np.linalg.pinv(S)
inv(S)

array([[  3837.58885062,  -8138.60069147,   4690.64729066],
       [ -8138.60069147,  17501.47304576, -10178.71687947],
       [  4690.64729066, -10178.71687947,   5960.2186682 ]])

Criterion function of optimal GMM with weighting matrix $W$:

$Q(\theta)  = \left(\frac{1}{n}\sum_{i=1}^{n}	{\bf h}(\theta,X_{i})\right)^{T}  W\left(\frac{1}{n}\sum_{i=1}^{n}	{\bf h}(\theta,X_{i})\right)$

In [301]:
def Q_optimal(theta,data,W):
  hx=0
  for x_data in data:
    hx += h(theta,x_data)/len(data)
  return np.matmul(np.matmul(np.transpose(hx),W),hx)[0][0]

Minimizing the GMM objective function with optimal weighting matrix

In [302]:
res = minimize_scalar(Q_optimal,args=(data,inv(S)))

In [303]:
res

     fun: 0.13244718114744697
    nfev: 20
     nit: 16
 success: True
       x: 1.1331002461152475

## More general example
More generally, suppose we observe the following data

In [384]:
np.random.seed(2)
data = np.random.uniform(0,2.5,(100,))

In [385]:
#Method of moments
np.mean(data)*2

2.300141226760969

In [386]:
#GMM with identity weighting matrix
res = minimize_scalar(Q,args = data)
res

     fun: 0.0004997826551992208
    nfev: 15
     nit: 10
 success: True
       x: 2.31325678529331

In [387]:
#Optimal GMM
theta_hat = res.x
S = 0
for x_data in data: 
  S += np.matmul(h(theta_hat,x_data),np.transpose(h(theta_hat,x_data)))/len(data)
inv(S)

array([[ 182.9620483 , -168.47873701,   42.53665462],
       [-168.47873701,  165.57519418,  -43.49207085],
       [  42.53665462,  -43.49207085,   11.74283067]])

In [388]:
res = minimize_scalar(Q_optimal,args=(data,inv(S)))
res

     fun: 0.03334602289517173
    nfev: 19
     nit: 14
 success: True
       x: 2.3913022568201696

## Asymptotic variance

$\hat{G} = \frac{1}{n}\sum_{i=1}^{n}	\frac{\partial {\bf h}(\theta,X_{i})}{\partial \theta}\Big|_{\theta = \hat{\theta}}$

In [389]:
def g(theta):
  return np.array([[-1/2],
            [-(2*theta)/3],
            [-3*(theta**2)/4]])

In [390]:
theta_ogmm = res.x

Asymptotic variance of optimal GMM is: $\frac{1}{n}(\hat{G}^{T} \hat{S}^{-1} \hat{G} )^{-1}$

In [391]:
(1/len(data))*(np.matmul(np.matmul(np.transpose(g(theta_ogmm)),inv(S)),g(theta_ogmm)))**-1

array([[0.00604387]])

In [ ]:
np.random.seed(2)
data = np.random.uniform(0,2.5,(1000,))